In [1]:
import re
import numpy as np
import pandas as pd
import scipy.sparse as ss
from bs4 import BeautifulSoup
import spacy
import spacy_fastlang
from spacy_cleaner import processing, Cleaner
from corextopic import corextopic as ct
from corextopic import vis_topic as vt
from gensim.corpora.dictionary import Dictionary
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from octis.evaluation_metrics.coherence_metrics import Coherence


def split_documents_by_words(documents, max_words=512):
    """
    Split documents if one document's word count is over than max_words.

    Args:
        documents (list): List of documents as strings.
        max_words (int): Maximum number of words for each document.

    Returns:
        list: List of split documents.
    """
    split_documents = []
    for doc in documents:
        words = doc.split()
        num_words = len(words)
        if num_words <= max_words:
            split_documents.append(doc)
        else:
            # Split document into segments of max_words
            num_segments = num_words // max_words
            for i in range(num_segments + 1):
                start_idx = i * max_words
                end_idx = (i + 1) * max_words
                if ' '.join(words[start_idx:end_idx]) != '' or ' '.join(words[start_idx:end_idx]) != ' ':
                    split_documents.append(' '.join(words[start_idx:end_idx]))
    return split_documents

df = pd.read_csv('/home/yy2046/Workspace/DCEE2023/datasets/twitter/twitter_junhao.csv', encoding='unicode_escape')
data = df['full_text']
    
''' preprocess '''
cleaned_data = []
model = spacy.load("en_core_web_sm")
cleaner = Cleaner( 
    model,
    processing.remove_stopword_token,
    processing.remove_punctuation_token,
    processing.remove_email_token,
    processing.remove_url_token,
    processing.mutate_lemma_token,
    
)

for html_text in data:
    soup = BeautifulSoup(html_text, 'html.parser')
    soup_text = soup.get_text().lower()
    cleaned_data.append(soup_text)
# print(cleaned_data[0])
print('spaCy preprocess start!')
cleaned_data = cleaner.clean(cleaned_data)
print(len(cleaned_data))


docs_list = split_documents_by_words(cleaned_data, max_words=512)

seed_topic_list = [["reduce"], ["reuse"], ["recycle"]]
vectorizer = CountVectorizer(stop_words='english',
                             max_features=20000,
                             binary=True)
search_space = {
    'anchor_strength': [1],
    'n_hidden': [5]
}

# gridSampler from optuna
sampler = optuna.samplers.GridSampler(search_space)

def objective(trial):
    anchor_strength = trial.suggest_float("anchor_strength", -100, 100)
    n_hidden = trial.suggest_int("n_hidden", -100, 100)

    doc_word = vectorizer.fit_transform(docs_list)
    doc_word = ss.csr_matrix(doc_word)
    words = list(np.asarray(vectorizer.get_feature_names_out()))
    topic_model = ct.Corex(n_hidden=n_hidden,
                        words=words,
                        verbose=False,
                        seed=42)
    topic_model.fit(doc_word,
                    words=words,
                    anchors=seed_topic_list,
                    anchor_strength=anchor_strength)
    topics = topic_model.get_topics()
    for n,topic in enumerate(topics):
        topic_words,_,_ = zip(*topic)
        print('{}: '.format(n) + ', '.join(topic_words))
    '''coherence computation'''
    corpus = [ doc.split(' ') for doc in cleaned_data]
    npmi = Coherence(texts=corpus, topk=10, measure='c_npmi')

    results = topic_model.get_topics()
    # print(results)
    extracted_words = [[item[0] for item in temp] for temp in results]

    # print(extracted_words)

    try:
        npmi_score = npmi.score({'topics':extracted_words})
    except:
        npmi_score = -99

    return npmi_score

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=len(search_space['anchor_strength']) * len(search_space['n_hidden']))

# print the best Params
best_trial = study.best_trial
print("Best trial:")
print(f"Value (NPMI): {best_trial.value}")
print("Params: ")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

/tmp/ipykernel_1384565/4122999602.py:61: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, 'html.parser')


spaCy preprocess start!


Cleaning Progress: 100%|██████████| 3922/3922 [00:06<00:00, 649.65it/s]
[I 2024-04-13 20:59:03,527] A new study created in memory with name: no-name-54aa522a-8aa8-4bf2-9964-e045897a2cdc


3922
0: reduce, emission, carbon, resource, waste, sustainable, planet, recyclability, sustainablepackage, co2
1: reuse, charity, item, repair, charitynee, monday, supportlocal, amazing, mondaythought, volunteer
2: recycle, recycling, tyre, tyrerecycle, battery, tirerecycle, tyrerecovery, recyclingbusiness, noexcusesgocircular, tire
3: forsale, register, event, cisco, april, conference, 2023, port, summit, join
4: breakfreefromplastic, wastetrade, baselconvention, wasteshipment, ban, endwastecolonialism, export, mess, tidy, eu


[I 2024-04-13 20:59:22,144] Trial 0 finished with value: 0.1537789943704965 and parameters: {'anchor_strength': 1.0, 'n_hidden': 5}. Best is trial 0 with value: 0.1537789943704965.


Best trial:
Value (NPMI): 0.1537789943704965
Params: 
anchor_strength: 1.0
n_hidden: 5
